In [1]:
#Base code: https://keras.io/examples/nlp/text_classification_with_transformer/

In [2]:
#! pip install pandas
#! pip install scikit-learn
#! pip install keras
#! pip install tensorflow


In [3]:
# General imports
import keras
import pandas as pd
import json
import tensorflow as tf
from collections import Counter
import numpy as np
import emoji
import string
import re

2024-06-29 03:30:16.155732: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-29 03:30:16.158697: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-29 03:30:16.195349: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 03:30:16.845612: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
#Trial train data
train_filepath = 'data/challenge_data/germeval-competition-traindev.jsonl' #'data/development_data/germeval-development-train.jsonl' 
test_filepath = 'data/challenge_data/germeval-competition-test.jsonl'  #'data/development_data/germeval-development-test.jsonl'
train_data = open(train_filepath).read()
test_data = open(test_filepath).read()
train_data[:100], test_data[:100]

('{"id": "a733e8a47708ce1d77060266d365e5b5", "text": "Wen man nicht reinl\\u00e4\\u00dft, den muss man a',
 '{"id": "f3b81af2f6852bf1b9896629525d2f41", "text": "Ja, Frauen k\\u00f6nnen krankhaft eifers\\u00fccht')

In [5]:
#json.loads(train_data)
json_train_data = [json.loads(jline) for jline in train_data.splitlines()]
json_test_data = [json.loads(jline) for jline in test_data.splitlines()]

numTrainSamples = len(json_train_data)
json_train_data[0], json_test_data[0]

({'id': 'a733e8a47708ce1d77060266d365e5b5',
  'text': 'Wen man nicht reinläßt, den muss man auch nicht integrieren.',
  'annotations': [{'user': 'A001', 'label': '0-Kein'},
   {'user': 'A002', 'label': '0-Kein'},
   {'user': 'A005', 'label': '0-Kein'},
   {'user': 'A008', 'label': '0-Kein'},
   {'user': 'A007', 'label': '0-Kein'},
   {'user': 'A004', 'label': '0-Kein'},
   {'user': 'A009', 'label': '0-Kein'},
   {'user': 'A003', 'label': '0-Kein'},
   {'user': 'A012', 'label': '0-Kein'},
   {'user': 'A010', 'label': '0-Kein'}]},
 {'id': 'f3b81af2f6852bf1b9896629525d2f41',
  'text': 'Ja, Frauen können krankhaft eifersüchtig werden.\nDa haben Sie wohl Recht.\n\nZumeist ist dann aber Gift das Mittel der Wahl.',
  'annotators': ['A001',
   'A002',
   'A005',
   'A008',
   'A007',
   'A004',
   'A009',
   'A003',
   'A010',
   'A012']})

In [6]:
len(json_train_data), len(json_test_data)

(5998, 1986)

In [7]:
df = pd.DataFrame(json_train_data+json_test_data)
#df.head()
X = list(df['text'])
df.tail()

,id,text,annotations,annotators
7979,2f7322c62b63ff74ec945bb38ed9f258,Ihre Partnerin schämt sich wahrscheinlich für ...,NaN,"[A002, A009, A010]"
7980,ec5fe35f542aac2f3155177dbf2731c2,glaube ich diese These einfach nicht.Grund: He...,NaN,"[A002, A009, A010]"
7981,6674986a02bab67b011df90cc7396a96,Damit die Ehefrau dann ein Dauervisum erhält m...,NaN,"[A002, A009, A010]"
7982,2a3774eba33afe18af2f0d312d081bb3,Ich selbst habe in meiner Hierarchie zwei Frau...,NaN,"[A002, A009, A010]"
7983,43891e25f522166dda98045a33563c05,Ich finde die Nahaufnahmen und sexistische Be...,NaN,"[A002, A009, A010]"


In [8]:
# Get the list of all annotator IDs
#annotator_ids = sorted(set([ann['user'] for annotations in df['annotations'] for ann in annotations]))


# Define the function to extract annotator IDs
def get_annotator_ids(train_filepath, test_filepath):
    train_data = open(train_filepath).read()
    json_train_data = [json.loads(jline) for jline in train_data.splitlines()]
    train_annotator_ids = sorted(set([ann['user'] for annotations in json_train_data for ann in annotations['annotations']]))

    test_data = open(test_filepath).read()
    json_test_data = [json.loads(jline) for jline in test_data.splitlines()]
    test_annotator_ids = sorted(set([annotator for annotations in json_test_data for annotator in annotations['annotators']]))

    # Combine and deduplicate annotator IDs from both train and test data
    annotator_ids = sorted(set(train_annotator_ids + test_annotator_ids))

    return annotator_ids

annotator_ids = get_annotator_ids(train_filepath, test_filepath)
annotator_ids

['A001',
 'A002',
 'A003',
 'A004',
 'A005',
 'A007',
 'A008',
 'A009',
 'A010',
 'A012']

In [9]:
# Define the feature extraction functions
def count_special_characters(text):
    return len(re.findall(r'[!@#$%^&*(),.?":{}|<>]', text))

def count_capitalized_words(text):
    return sum(1 for word in text.split() if word.isupper())

def count_emojis(text):
    return len(emoji.emoji_list(text))

def count_characters(text):
    return len(text)

def count_words(text):
    return len(text.split())

def count_punctuations(text):
    return sum(1 for char in text if char in string.punctuation)

# Define the function to process data
def get_additional_features(filepath, annotator_ids,annotations_key):
    # Load data
    data = open(filepath).read()
    json_data = [json.loads(jline) for jline in data.splitlines()]
    df = pd.DataFrame(json_data)
    X = list(df['text'])

    # Initialize a list to store binary vectors for each document
    annotator_presence = []

    # Initialize a list to store binary vectors for each document
    annotator_presence = []

    # Iterate over the annotations for each document
    for annotations in df[annotations_key]:
        # Initialize a binary vector for the current document
        doc_vector = [0] * len(annotator_ids)
        # Mark the presence of each annotator
        for ann in annotations:
            user = ann if annotations_key == 'annotators' else ann['user']
            idx = annotator_ids.index(user)
            doc_vector[idx] = 1
        # Append the binary vector to the list
        annotator_presence.append(doc_vector)

    # Convert the list of binary vectors into a numpy array
    annotator_presence_matrix = np.array(annotator_presence)

    # Apply the feature extraction functions to your dataset
    df['special_char_count'] = df['text'].apply(count_special_characters)
    df['capitalized_word_count'] = df['text'].apply(count_capitalized_words)
#    df['emoji_count'] = df['text'].apply(count_emojis)
    df['char_count'] = df['text'].apply(count_characters)
    df['word_count'] = df['text'].apply(count_words)
    df['punctuation_count'] = df['text'].apply(count_punctuations)

    # Convert these features to numpy arrays
    special_char_count = np.array(df['special_char_count']).reshape(-1, 1)
    capitalized_word_count = np.array(df['capitalized_word_count']).reshape(-1, 1)
#    emoji_count = np.array(df['emoji_count']).reshape(-1, 1)
    char_count = np.array(df['char_count']).reshape(-1, 1)
    word_count = np.array(df['word_count']).reshape(-1, 1)
    punctuation_count = np.array(df['punctuation_count']).reshape(-1, 1)
    
    # Combine these features into a single array
    additional_features = np.hstack([annotator_presence_matrix, special_char_count, capitalized_word_count, #emoji_count,
                                     char_count, word_count, punctuation_count])

    return additional_features



In [10]:
# Usage
#train_filepath = 'data/development_data/germeval-development-train.jsonl'
#test_filepath = 'data/development_data/germeval-development-test.jsonl'
# Process training data
additional_features = get_additional_features(train_filepath, annotator_ids, 'annotations')
additional_features_test = get_additional_features(test_filepath, annotator_ids, 'annotators')

In [11]:
'''
#defining additional features
def count_special_characters(text):
    return len(re.findall(r'[!@#$%^&*(),.?":{}|<>]', text))

def count_capitalized_words(text):
    return sum(1 for word in text.split() if word.isupper())

def count_emojis(text):
    return len(emoji.emoji_list(text))

def count_characters(text):
    return len(text)

def count_words(text):
    return len(text.split())

def count_punctuations(text):
    return sum(1 for char in text if char in string.punctuation)


# Apply the feature extraction functions to your dataset
df['special_char_count'] = df['text'].apply(count_special_characters)
df['capitalized_word_count'] = df['text'].apply(count_capitalized_words)
df['emoji_count'] = df['text'].apply(count_emojis)
df['char_count'] = df['text'].apply(count_characters)
df['word_count'] = df['text'].apply(count_words)
df['punctuation_count'] = df['text'].apply(count_punctuations)

# Convert these features to numpy arrays
special_char_count = np.array(df['special_char_count']).reshape(-1, 1)
capitalized_word_count = np.array(df['capitalized_word_count']).reshape(-1, 1)
emoji_count = np.array(df['emoji_count']).reshape(-1, 1)
char_count = np.array(df['char_count']).reshape(-1, 1)
word_count = np.array(df['word_count']).reshape(-1, 1)
punctuation_count = np.array(df['punctuation_count']).reshape(-1, 1)

# Combine these features into a single array
#additional_features = np.hstack([annotator_presence_matrix,special_char_count, capitalized_word_count, emoji_count,
#                                 char_count, word_count, punctuation_count])
'''

'\n#defining additional features\ndef count_special_characters(text):\n    return len(re.findall(r\'[!@#$%^&*(),.?":{}|<>]\', text))\n\ndef count_capitalized_words(text):\n    return sum(1 for word in text.split() if word.isupper())\n\ndef count_emojis(text):\n    return len(emoji.emoji_list(text))\n\ndef count_characters(text):\n    return len(text)\n\ndef count_words(text):\n    return len(text.split())\n\ndef count_punctuations(text):\n    return sum(1 for char in text if char in string.punctuation)\n\n\n# Apply the feature extraction functions to your dataset\ndf[\'special_char_count\'] = df[\'text\'].apply(count_special_characters)\ndf[\'capitalized_word_count\'] = df[\'text\'].apply(count_capitalized_words)\ndf[\'emoji_count\'] = df[\'text\'].apply(count_emojis)\ndf[\'char_count\'] = df[\'text\'].apply(count_characters)\ndf[\'word_count\'] = df[\'text\'].apply(count_words)\ndf[\'punctuation_count\'] = df[\'text\'].apply(count_punctuations)\n\n# Convert these features to numpy arr

In [12]:
#y_train = [0 if label == 0 else 1 for label in labels]
#y_train[:13]
# Initialize label lists
dictlabels = {'bin_maj': [], 'bin_one': [], 'bin_all': [], 'multi_maj': [], 'disagree_bin': [], 'dist_bin_0': [], 'dist_bin_1': [], 'dist_multi_0': [],
            'dist_multi_1': [], 'dist_multi_2': [], 'dist_multi_3': [], 'dist_multi_4': []}

for annotations in df['annotations'][:numTrainSamples]:
    #print(annotations)
    labels = [ann['label'] for ann in annotations]
    # Count occurrences of each label
    label_count = Counter(labels)
    # Calculate the total number of annotations
    total_annotations = sum(label_count.values())
    # bin_maj
    if label_count['0-Kein'] > sum(label_count.values()) / 2: #TK: Why /2? label_count.values() should be without label_count['0-kein'], use >= for one
        dictlabels['bin_maj'].append(0)
    else:
        dictlabels['bin_maj'].append(1)  # If there was no majority, consider both 1 and 0 as correct

    # bin_one
    if any(label != '0-Kein' for label in labels): #TK: bin_one and bin_all - the code seems similar
        dictlabels['bin_one'].append(1)
    else:
        dictlabels['bin_one'].append(0)

    # bin_all
    if all(label != '0-Kein' for label in labels):
        dictlabels['bin_all'].append(1)
    else:
        dictlabels['bin_all'].append(0)

    # multi_maj
    #dictlabels['multi_maj'].append(label_count.most_common(1)[0][0][0])  # Append a list with a single integer
    # multi_maj
    dict_label_count = {'0-Kein': labels.count('0-Kein'), '1-Gering': labels.count('1-Gering'), 
                        '2-Vorhanden': labels.count('2-Vorhanden'), '3-Stark': labels.count('3-Stark'), '4-Extrem': labels.count('4-Extrem')}
    maxVal = 0
    lbl = ''
    for k,v in dict_label_count.items():
        if v >= maxVal:
            maxVal = v
            lbl = k
    if lbl == '0-Kein':
        dictlabels['multi_maj'].append([1, 0, 0, 0, 0]) 
    elif lbl == '1-Gering':
        dictlabels['multi_maj'].append([0, 1, 0, 0, 0])
    elif lbl == '2-Vorhanden':
        dictlabels['multi_maj'].append([0, 0, 1, 0, 0])
    elif lbl == '3-Stark':
        dictlabels['multi_maj'].append([0, 0, 0, 1, 0])
    elif lbl == '4-Extrem':
        dictlabels['multi_maj'].append([0, 0, 0, 0, 1])

    # disagree_bin
    if len(label_count) > 1 and '0-Kein' in label_count:
        dictlabels['disagree_bin'].append(1)
    else:
        dictlabels['disagree_bin'].append(0)

    # Calculate dist_bin
    dictlabels['dist_bin_0'].append(label_count['0-Kein'] / total_annotations)
    dictlabels['dist_bin_1'].append((total_annotations - label_count['0-Kein']) / total_annotations)
    
    # Calculate dist_multi
    dictlabels['dist_multi_0'].append(label_count['0-Kein'] / total_annotations)
    dictlabels['dist_multi_1'].append(label_count['1-Gering'] / total_annotations)
    dictlabels['dist_multi_2'].append(label_count['2-Vorhanden'] / total_annotations)
    dictlabels['dist_multi_3'].append(label_count['3-Stark'] / total_annotations)
    dictlabels['dist_multi_4'].append(label_count['4-Extrem'] / total_annotations) 

In [13]:
#y_multi_maj = np.array([multi_maj_mapping[label[0]] if isinstance(label, list) else multi_maj_mapping[label] for label in df['multi_maj']])

# Extract the new target variables as arrays
y_bin_maj = np.array(dictlabels['bin_maj'])
y_bin_one = np.array(dictlabels['bin_one'])
y_bin_all = np.array(dictlabels['bin_all'])
y_multi_maj = np.array(dictlabels['multi_maj'])
y_disagree_bin = np.array(dictlabels['disagree_bin'])
y_dist_bin_0 = np.array(dictlabels['dist_bin_0'])
y_dist_bin_1 = np.array(dictlabels['dist_bin_1'])
y_dist_multi_0 = np.array(dictlabels['dist_multi_0'])
y_dist_multi_1 = np.array(dictlabels['dist_multi_1'])
y_dist_multi_2 = np.array(dictlabels['dist_multi_2'])
y_dist_multi_3 = np.array(dictlabels['dist_multi_3'])
y_dist_multi_4 = np.array(dictlabels['dist_multi_4'])


In [14]:
#y_multi_maj[:20]

In [15]:
# Combine the target variables into a single array if needed
y_train_combined = np.column_stack([y_bin_maj, y_bin_one, y_bin_all, y_multi_maj, y_disagree_bin,
                                    y_dist_bin_0, y_dist_bin_1, y_dist_multi_0, y_dist_multi_1, y_dist_multi_2, y_dist_multi_3, y_dist_multi_4])


In [16]:
#Output label distribution analysis
print('bin_maj (0, 1): ', (y_bin_maj == 0).sum()*100//len(y_bin_maj),'%, ', (y_bin_maj == 1).sum()*100//len(y_bin_maj), '%')
print('bin_one (0, 1): ', (y_bin_one == 0).sum()*100//len(y_bin_one),'%, ', (y_bin_one == 1).sum()*100//len(y_bin_one), '%')
print('bin_all (0, 1): ', (y_bin_all == 0).sum()*100//len(y_bin_all),'%, ', (y_bin_all == 1).sum()*100//len(y_bin_all), '%')
tmp = [0, 0, 0, 0, 0]
for item in y_multi_maj:
    tmp[0] += item[0];    tmp[1] += item[1];     tmp[2] += item[2];    tmp[3] += item[3];    tmp[4] += item[4]
print('multi_maj %: ',tmp[0]*100//sum(tmp), tmp[1]*100//sum(tmp), tmp[2]*100//sum(tmp), tmp[3]*100//sum(tmp), tmp[4]*100//sum(tmp))

bin_maj (0, 1):  65 %,  34 %
bin_one (0, 1):  46 %,  53 %
bin_all (0, 1):  83 %,  16 %
multi_maj %:  67 3 15 10 1


In [17]:
#Text to sequence of words
X_seq = []
for item in X:
    word_seq = tf.keras.preprocessing.text.text_to_word_sequence(
        item,
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
        lower=True,
        split=' '
    )
    X_seq.append(word_seq)
len(X_seq)

7984

In [18]:
# this integer encoding is purely based on position, you can do this in other ways
#integer_mapping = {}

# Flatten the list of word sequences to get a list of all words
all_words = [word for seq in X_seq for word in seq]

# Create a mapping from each unique word to a unique integer
unique_words = set(all_words)
integer_mapping = {word: i for i, word in enumerate(unique_words)}

# Convert each word sequence to a sequence of integers using the mapping
X_int_seq = [[integer_mapping[word] for word in seq] for seq in X_seq]

# Example outputs
print(len(X_int_seq))  # Original word sequences
len(integer_mapping)

7984


32362

In [19]:
wcount = []
for seq in X_seq:
    wcount.append(len(seq))
sum(wcount)/len(wcount), max(wcount), min(wcount)

(32.862850701402806, 173, 0)

In [20]:

# Model Architecture imports
#from tensorflow.python import ops
#from keras import ops
from keras import layers
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.layers import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [21]:
vocab_size = len(integer_mapping)  # considering all vocabulary
maxlen = 80  # Only consider the first 60 words - max is about 100, average 33.158

# Load and preprocess the data
#(x_train, y_train), (x_val, y_val) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)
#x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
#x_val = tf.keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

# Convert the integer sequences to numpy arrays and pad them
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_int_seq, maxlen=maxlen)


In [22]:
# Example usage
#num_heads = 2  # Number of attention heads
#ff_dim = 32  # Hidden layer size in feed forward network inside transformer

# Load pretrained FastText embeddings
fasttext_path = 'pretrained_embeddings/cc.de.300.vec'
embedding_dim = 300

embeddings_index = {}
with open(fasttext_path, encoding='utf-8') as f:
    for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [23]:
#vocab_size = len(unique_words)
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in integer_mapping.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [24]:
embedding_matrix.shape

(32362, 300)

In [25]:
# Create an Embedding layer with pretrained weights
embedding_dim = 300
embedding_layer = layers.Embedding(input_dim=vocab_size,
                                   output_dim=embedding_dim,
                                   weights=[embedding_matrix],
                                   input_length=maxlen,
                                   trainable=False)  # Set to True to fine-tune the embeddings


/home/khantr/.local/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-06-29 03:31:52.193041: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-29 03:31:52.193536: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [26]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    #def call(self, inputs, training):
    def call(self, inputs, training=None, **kwargs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [27]:
# Model 1 - Architecture

num_heads = 2  # Number of attention heads
ff_dim = 30  # Hidden layer size in feed forward network inside transformer

#Model Architecture
#inputs = layers.Input(shape=(maxlen,))
# Define input layers for the text and additional features
text_input = layers.Input(shape=(maxlen,))
features_input = layers.Input(shape=(additional_features.shape[1],))
x = embedding_layer(text_input)
transformer_block = TransformerBlock(embedding_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)

# Concatenate the processed text with additional features
x = layers.Concatenate()([x, features_input])
x = layers.Dropout(0.3)(x)

x = layers.Dense(1000, activation="leaky_relu")(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(100, activation="leaky_relu")(x)
x = layers.Dropout(0.2)(x)
#x = layers.Dense(50, activation = "relu")(x)
#outputs = layers.Dense(2, activation="softmax")(x)


# Separate output layers for each component of the target variable
output_bin_maj = layers.Dense(1, activation="sigmoid", name="output_bin_maj")(x)
output_bin_one = layers.Dense(1, activation="sigmoid", name="output_bin_one")(x)
output_bin_all = layers.Dense(1, activation="sigmoid", name="output_bin_all")(x)
#output_multi_maj = layers.Dense(5, activation="softmax", name="output_multi_maj")(x)  # Assuming the multi-majority is a multi-class problem
output_disagree_bin = layers.Dense(1, activation="sigmoid", name="output_disagree_bin")(x)
output_dist_bin_0 = layers.Dense(1, activation="sigmoid", name="output_dist_bin_0")(x)
output_dist_bin_1 = layers.Dense(1, activation="sigmoid", name="output_dist_bin_1")(x)
output_dist_multi_0 = layers.Dense(1, activation="sigmoid", name="output_dist_multi_0")(x)
output_dist_multi_1 = layers.Dense(1, activation="sigmoid", name="output_dist_multi_1")(x)
output_dist_multi_2 = layers.Dense(1, activation="sigmoid", name="output_dist_multi_2")(x)
output_dist_multi_3 = layers.Dense(1, activation="sigmoid", name="output_dist_multi_3")(x)
output_dist_multi_4 = layers.Dense(1, activation="sigmoid", name="output_dist_multi_4")(x)


#model = tf.keras.Model(inputs=inputs, outputs=outputs)
model = tf.keras.Model(inputs=[text_input,features_input], outputs=[output_bin_maj, output_bin_one, output_bin_all, #output_multi_maj, 
                                                                    output_disagree_bin,
                                               output_dist_bin_0, output_dist_bin_1, output_dist_multi_0, output_dist_multi_1,
                                               output_dist_multi_2, output_dist_multi_3, output_dist_multi_4])
#model.summary()

In [28]:
#model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizer="adamW",
              loss={"output_bin_maj": "binary_focal_crossentropy", #"binary_crossentropy", 
                    "output_bin_one": "binary_focal_crossentropy",
                    "output_bin_all": "binary_focal_crossentropy",
                    #"output_multi_maj": "categorical_focal_crossentropy", #"categorical_crossentropy",
                    "output_disagree_bin": "binary_focal_crossentropy",
                    "output_dist_bin_0": "mse",
                    "output_dist_bin_1": "mse",
                    "output_dist_multi_0": "mse",
                    "output_dist_multi_1": "mse",
                    "output_dist_multi_2": "mse",
                    "output_dist_multi_3": "mse",
                    "output_dist_multi_4": "mse"},
              metrics={"output_bin_maj": "binary_accuracy", 
                       "output_bin_one": "binary_accuracy",
                       "output_bin_all": "binary_accuracy",
                       #"output_multi_maj": "categorical_accuracy",
                       "output_disagree_bin": "binary_accuracy",
                       "output_dist_bin_0": "mape",
                       "output_dist_bin_1": "mape",
                       "output_dist_multi_0": "mape",
                       "output_dist_multi_1": "mape",
                       "output_dist_multi_2": "mape",
                       "output_dist_multi_3": "mape",
                       "output_dist_multi_4": "mape"})


In [29]:
X_train_padded = X_padded[:numTrainSamples]
X_test_padded = X_padded[numTrainSamples:]
numTrainSamples

5998

In [30]:
### Train the model
history = model.fit([X_train_padded,additional_features], {"output_bin_maj": y_bin_maj,
                                     "output_bin_one": y_bin_one,
                                     "output_bin_all": y_bin_all,
                                     #"output_multi_maj": y_multi_maj,
                                     "output_disagree_bin": y_disagree_bin,
                                     "output_dist_bin_0": y_dist_bin_0,
                                     "output_dist_bin_1": y_dist_bin_1,
                                     "output_dist_multi_0": y_dist_multi_0,
                                     "output_dist_multi_1": y_dist_multi_1,
                                     "output_dist_multi_2": y_dist_multi_2,
                                     "output_dist_multi_3": y_dist_multi_3,
                                     "output_dist_multi_4": y_dist_multi_4},
                                     #"output_multi_maj": np.array(y_multi_maj)},
                    #sample_weight=sample_weights_multi_maj,
                    batch_size=40, epochs=5, validation_split=0.2, shuffle = True)

Epoch 1/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 8.9653 - output_bin_all_binary_accuracy: 0.7099 - output_bin_maj_binary_accuracy: 0.5567 - output_bin_one_binary_accuracy: 0.4937 - output_disagree_bin_binary_accuracy: 0.5358 - output_dist_bin_0_mape: 134405312.0000 - output_dist_bin_1_mape: 138979184.0000 - output_dist_multi_0_mape: 91743088.0000 - output_dist_multi_1_mape: 103356208.0000 - output_dist_multi_2_mape: 96827640.0000 - output_dist_multi_3_mape: 94575400.0000 - output_dist_multi_4_mape: 131812984.0000 - val_loss: 3.1377 - val_output_bin_all_binary_accuracy: 0.8408 - val_output_bin_maj_binary_accuracy: 0.5692 - val_output_bin_one_binary_accuracy: 0.3883 - val_output_disagree_bin_binary_accuracy: 0.5167 - val_output_dist_bin_0_mape: 146157888.0000 - val_output_dist_bin_1_mape: 45868972.0000 - val_output_dist_multi_0_mape: 141338944.0000 - val_output_dist_multi_1_mape: 13644878.0000 - val_output_dist_multi_2_mape: 23181416.0000 - val_output_dist_multi_3_mape: 27548

In [31]:
text_input = layers.Input(shape=(maxlen,))
features_input = layers.Input(shape=(additional_features.shape[1],))
x = embedding_layer(text_input)
transformer_block = TransformerBlock(embedding_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)

# Concatenate the processed text with additional features
x = layers.Concatenate()([x, features_input])
x = layers.Dropout(0.3)(x)

x = layers.Dense(1000, activation="leaky_relu")(x)
x = layers.Dropout(0.3)(x)

x = layers.Dense(100, activation="leaky_relu")(x)
x = layers.Dropout(0.2)(x)
#x = layers.Dense(50, activation = "relu")(x)
#outputs = layers.Dense(2, activation="softmax")(x)


# Separate output layers for each component of the target variable
output_multi_maj = layers.Dense(5, activation="softmax", name="output_multi_maj")(x)  # Assuming the multi-majority is a multi-class problem
output_disagree_bin = layers.Dense(1, activation="sigmoid", name="output_disagree_bin")(x)

#model = tf.keras.Model(inputs=inputs, outputs=outputs)
model2 = tf.keras.Model(inputs=[text_input,features_input], outputs=[output_multi_maj, output_disagree_bin])
#model.summary()
#model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
model2.compile(optimizer="adamW",
              loss={"output_multi_maj": "categorical_focal_crossentropy", #"categorical_crossentropy",
                   "output_disagree_bin": "binary_focal_crossentropy"},
              metrics={"output_multi_maj": "categorical_accuracy",
                      "output_disagree_bin" : "binary_accuracy"})
### Train the model
history2 = model2.fit([X_train_padded,additional_features], {"output_multi_maj": y_multi_maj, "output_disagree_bin" : y_disagree_bin},
                                     #"output_multi_maj": np.array(y_multi_maj)},
                    #sample_weight=sample_weights_multi_maj,
                    batch_size=40, epochs=5, validation_split=0.2, shuffle = True)

Epoch 1/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - loss: 4.1450 - output_disagree_bin_binary_accuracy: 0.5355 - output_multi_maj_categorical_accuracy: 0.5279 - val_loss: 0.5644 - val_output_disagree_bin_binary_accuracy: 0.5450 - val_output_multi_maj_categorical_accuracy: 0.6508
Epoch 2/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - loss: 1.1726 - output_disagree_bin_binary_accuracy: 0.5392 - output_multi_maj_categorical_accuracy: 0.5673 - val_loss: 0.7783 - val_output_disagree_bin_binary_accuracy: 0.5483 - val_output_multi_maj_categorical_accuracy: 0.6508
Epoch 3/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - loss: 0.9077 - output_disagree_bin_binary_accuracy: 0.5714 - output_multi_maj_categorical_accuracy: 0.5941 - val_loss: 0.3807 - val_output_disagree_bin_binary_accuracy: 0.5483 - val_output_multi_maj_categorical_accuracy: 0.6508
Epoch 4/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - loss: 0.6527 - output_disagree_bin_binary_accuracy: 0.5519 - output_multi_maj_categorical_accuracy:

In [32]:
model.save("saved_models/development_model4_final.keras")

In [ ]:
#loaded_model = keras.saving.load_model("saved_models/trial_model.keras")

Test Model

In [ ]:
[X_test_padded,additional_features_test][0]

In [38]:
ypred2[1][3]

array([0.42225596], dtype=float32)

In [32]:
ypred = model.predict([X_test_padded,additional_features_test], batch_size=None, verbose='auto', steps=None, callbacks=None)

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step


In [33]:
ypred2 = model2.predict([X_test_padded,additional_features_test], batch_size=None, verbose='auto', steps=None, callbacks=None)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step


In [83]:
len(ypred[0])

1986

In [ ]:
#Output to DF
test_id = [json_test_data[i]['id'] for i in range(len(json_test_data))]
#ypred_bin_maj = [ypred[0][i][0] for i in range(len(ypred[0]))]
ypred_bin_maj = [1 if ypred[0][i][0] >= 0.5 else 0 for i in range(len(ypred[0]))]
ypred_bin_one = [1 if ypred[1][i][0] >= 0.5 else 0 for i in range(len(ypred[1]))]
ypred_bin_all = [1 if ypred[2][i][0] >= 0.5 else 0 for i in range(len(ypred[2]))]
categories = ["0-Kein", "1-Gering", "2-Vorhanden", "3-Stark", "4-Extrem"]
tmp = [categories[ll.index(max(ll))] for ll in [list(ypred[0][i]) for i in range(len(ypred[3]))]]

# Predictions for multi_maj
ypred_multi_maj_raw = tmp  # This is the raw output from the model for multi_maj
print(ypred_multi_maj_raw[0])

ypred_disagree_bin = [1 if ypred[4][i][0] >= 0.5 else 0 for i in range(len(ypred[4]))]

ypred_dist_bin_0 = [ypred[5][i][0] for i in range(len(ypred[5]))]
ypred_dist_bin_1 = [round(ypred[6][i][0],4) for i in range(len(ypred[6]))]
for i in range(len(ypred_dist_bin_0)):
    tt = ypred_dist_bin_0[i] + ypred_dist_bin_1[i]
    ypred_dist_bin_0[i] = ypred_dist_bin_0[i] / tt
    ypred_dist_bin_1[i] = ypred_dist_bin_1[i] / tt

ypred_dist_multi_0 = [ypred[7][i][0] for i in range(len(ypred[7]))]
ypred_dist_multi_1 = [ypred[8][i][0] for i in range(len(ypred[8]))]
ypred_dist_multi_2 = [ypred[9][i][0] for i in range(len(ypred[9]))]
ypred_dist_multi_3 = [ypred[10][i][0] for i in range(len(ypred[10]))]
ypred_dist_multi_4 = [ypred[11][i][0] for i in range(len(ypred[11]))]
for i in range(len(ypred_dist_multi_0)):
    tt = ypred_dist_multi_0[i] + ypred_dist_multi_1[i] + ypred_dist_multi_2[i] + ypred_dist_multi_3[i] + ypred_dist_multi_4[i]  
    ypred_dist_multi_0[i] = ypred_dist_multi_0[i] / tt
    ypred_dist_multi_1[i] = ypred_dist_multi_1[i] / tt
    ypred_dist_multi_2[i] = ypred_dist_multi_2[i] / tt
    ypred_dist_multi_3[i] = ypred_dist_multi_3[i] / tt
    ypred_dist_multi_4[i] = ypred_dist_multi_4[i] / tt

In [49]:
#ypred_multi_maj_raw

In [3]:
#! pip install tf-keras

In [85]:
df_output_subtask1 = pd.DataFrame({
              'id': test_id,
             'bin_maj': ypred_bin_maj,
             'bin_one': ypred_bin_one,
              'bin_all': ypred_bin_all,
              'multi_maj': ypred_multi_maj_raw,
              'disagree_bin': ypred_disagree_bin,
             })
df_output_subtask1.head(2)
df_output_subtask2 = pd.DataFrame({
                    'id': test_id,
                  'dist_bin_0': ypred_dist_bin_0,
              'dist_bin_1': ypred_dist_bin_1,
              'dist_multi_0': ypred_dist_multi_0,
              'dist_multi_1': ypred_dist_multi_1,
              'dist_multi_2': ypred_dist_multi_2,
              'dist_multi_3': ypred_dist_multi_3,
              'dist_multi_4': ypred_dist_multi_4

})

In [86]:
type(df_output_subtask2['dist_multi_1'])

pandas.core.series.Series

In [87]:
# Closed subtask 1 output file
subtask1_output = df_output_subtask1.to_csv('data/challenge_data/subtask1_output_model2.tsv', sep="\t", index = False) 
subtask2_output = df_output_subtask2.to_csv('data/challenge_data/subtask2_output_model2.tsv', sep="\t", index = False) 